In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from datetime import datetime as dt
import itertools


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm


from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

from keras.utils import np_utils

%matplotlib inline

Using TensorFlow backend.


# Load the Dataset

In [2]:
df = pd.read_csv("../datasets/FIFA 1990-2018 - Cleaned.csv")
squads = pd.read_csv("../datasets/New_data/2018 FIFA World Cup Squads.csv")
fifa18 = pd.read_csv("../datasets/New_data/complete.csv")
distance = pd.read_csv("../datasets/New_data/distance.csv", error_bad_lines=False)
fifa_ranking = pd.read_csv("../datasets/New_data/fifa_ranking.csv")

b'Skipping line 31: expected 4 fields, saw 5\nSkipping line 63: expected 4 fields, saw 5\nSkipping line 85: expected 4 fields, saw 5\nSkipping line 102: expected 4 fields, saw 5\nSkipping line 173: expected 4 fields, saw 5\nSkipping line 174: expected 4 fields, saw 5\nSkipping line 196: expected 4 fields, saw 5\nSkipping line 203: expected 4 fields, saw 5\nSkipping line 204: expected 4 fields, saw 5\nSkipping line 207: expected 4 fields, saw 5\nSkipping line 211: expected 4 fields, saw 5\nSkipping line 213: expected 4 fields, saw 5\n'


# Add Ranking for 2018

In [3]:
ranking_2018 = fifa_ranking[(fifa_ranking['rank_date']>"2018-04-01") & (fifa_ranking['rank_date']<"2018-05-01")]
ranking_2018.shape

(211, 17)

In [4]:
ranking_2018

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date,Unnamed: 16
57160,1,Germany,GER,1532.68,1609.0,0,784.89,784.89,804.19,402.09,463.51,139.05,1033.19,206.64,UEFA,2018-04-12,NaN
57161,2,Brazil,BRA,1384.24,1489.0,0,560.52,560.52,990.59,495.29,565.90,169.77,793.26,158.65,CONMEBOL,2018-04-12,NaN
57162,3,Belgium,BEL,1345.65,1337.0,2,684.48,684.48,594.99,297.50,592.33,177.70,929.90,185.98,UEFA,2018-04-12,NaN
57163,4,Portugal,POR,1306.35,1360.0,-1,687.22,687.22,658.47,329.23,576.48,172.94,584.73,116.95,UEFA,2018-04-12,NaN
57164,5,Argentina,ARG,1253.61,1359.0,-1,418.94,418.94,839.74,419.87,769.42,230.83,919.87,183.97,CONMEBOL,2018-04-12,NaN
57165,6,Switzerland,SUI,1179.21,1197.0,2,594.33,594.33,716.99,358.49,390.85,117.26,545.66,109.13,UEFA,2018-04-12,NaN
57166,7,France,FRA,1166.42,1185.0,2,517.68,517.68,828.04,414.02,347.19,104.16,652.83,130.57,UEFA,2018-04-12,NaN
57167,8,Spain,ESP,1161.59,1228.0,-2,571.55,571.55,593.19,296.59,684.65,205.39,440.29,88.06,UEFA,2018-04-12,NaN
57168,9,Chile,CHI,1145.74,1161.0,1,435.51,435.51,714.45,357.22,850.55,255.17,489.25,97.85,CONMEBOL,2018-04-12,NaN
57169,10,Poland,POL,1117.94,1228.0,-4,499.90,499.90,764.51,382.25,450.51,135.15,503.18,100.64,UEFA,2018-04-12,NaN


In [5]:
# Create new columns with ranking position of each team
df.insert(2, 'rank_Team1', df['Team1'].map(ranking_2018.set_index('country_full')['rank']))
df.insert(3, 'rank_Team2', df['Team2'].map(ranking_2018.set_index('country_full')['rank']))

df.insert(4, 'rank_change_Team1', df['Team1'].map(ranking_2018.set_index('country_full')['rank_change']))
df.insert(5, 'rank_change_Team2', df['Team2'].map(ranking_2018.set_index('country_full')['rank_change']))

In [6]:
df.head()

,Team1,Team2,rank_Team1,rank_Team2,rank_change_Team1,rank_change_Team2,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,66.0,70.0,-3.0,-1.0,Group Phase,Luzhniki Stadium,Moscow,18:00,14,6,2018,5,0,NaN
1,Egypt,Uruguay,46.0,17.0,-2.0,5.0,Group Phase,Ekaterinburg Arena,Ekaterinburg,17:00,15,6,2018,0,1,NaN
2,Morocco,IR Iran,42.0,36.0,0.0,-3.0,Group Phase,Saint Petersburg Stadium,St. Petersburg,18:00,15,6,2018,0,1,NaN
3,Portugal,Spain,4.0,8.0,-1.0,-2.0,Group Phase,Fisht Stadium,Sochi,21:00,15,6,2018,3,3,{WinTeamName} win on aggregate after regular t...
4,France,Australia,7.0,40.0,2.0,-3.0,Group Phase,Kazan Arena,Kazan,13:00,16,6,2018,2,1,NaN


# Add Team features

In [7]:
# AVG Age of each team in each position
team_feature=squads.groupby(["Team","Position"])['Age'].sum()/squads.groupby(["Team","Position"])['Age'].size()

In [8]:
team_feature=team_feature.to_frame()

In [9]:
team_feature.head()

Age
Team      Position           
Argentina DF        29.111111
          FW        28.500000
          GK        33.000000
          MF        26.428571
Australia DF        27.428571

In [10]:
# #AVG Age of each team in each position
age_avg=squads.groupby(["Team","Position"])['Age'].sum()/squads.groupby(["Team","Position"])['Age'].size()
age_avg = age_avg.to_frame()

age_avg['Team_name'] = age_avg.index.get_level_values(0).tolist()
age_avg['pos'] = age_avg.index.get_level_values(1).tolist()

In [11]:
age_avg.head()

Age  Team_name pos
Team      Position                          
Argentina DF        29.111111  Argentina  DF
          FW        28.500000  Argentina  FW
          GK        33.000000  Argentina  GK
          MF        26.428571  Argentina  MF
Australia DF        27.428571  Australia  DF

In [12]:
df.head()

,Team1,Team2,rank_Team1,rank_Team2,rank_change_Team1,rank_change_Team2,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,66.0,70.0,-3.0,-1.0,Group Phase,Luzhniki Stadium,Moscow,18:00,14,6,2018,5,0,NaN
1,Egypt,Uruguay,46.0,17.0,-2.0,5.0,Group Phase,Ekaterinburg Arena,Ekaterinburg,17:00,15,6,2018,0,1,NaN
2,Morocco,IR Iran,42.0,36.0,0.0,-3.0,Group Phase,Saint Petersburg Stadium,St. Petersburg,18:00,15,6,2018,0,1,NaN
3,Portugal,Spain,4.0,8.0,-1.0,-2.0,Group Phase,Fisht Stadium,Sochi,21:00,15,6,2018,3,3,{WinTeamName} win on aggregate after regular t...
4,France,Australia,7.0,40.0,2.0,-3.0,Group Phase,Kazan Arena,Kazan,13:00,16,6,2018,2,1,NaN


In [18]:
df.insert(2, 'Team1_avg_age_DF', df['Team1'].map(team_age_DF.set_index('Team_name')['Age']))

In [19]:
df.head()

,Team1,Team2,Team1_avg_age_DF,rank_Team1,rank_Team2,rank_change_Team1,rank_change_Team2,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,29.857143,66.0,70.0,-3.0,-1.0,Group Phase,Luzhniki Stadium,Moscow,18:00,14,6,2018,5,0,NaN
1,Egypt,Uruguay,28.750000,46.0,17.0,-2.0,5.0,Group Phase,Ekaterinburg Arena,Ekaterinburg,17:00,15,6,2018,0,1,NaN
2,Morocco,IR Iran,27.000000,42.0,36.0,0.0,-3.0,Group Phase,Saint Petersburg Stadium,St. Petersburg,18:00,15,6,2018,0,1,NaN
3,Portugal,Spain,28.375000,4.0,8.0,-1.0,-2.0,Group Phase,Fisht Stadium,Sochi,21:00,15,6,2018,3,3,{WinTeamName} win on aggregate after regular t...
4,France,Australia,24.375000,7.0,40.0,2.0,-3.0,Group Phase,Kazan Arena,Kazan,13:00,16,6,2018,2,1,NaN


In [16]:
team_age_DF = age_avg.loc[age_avg['pos'] == 'DF']

In [20]:
team_age_DF.head(32)

,,Age,Team_name,pos
Team,Position,,,
Argentina,DF,29.111111,Argentina,DF
Australia,DF,27.428571,Australia,DF
Belgium,DF,28.571429,Belgium,DF
Brazil,DF,29.750000,Brazil,DF
Colombia,DF,26.142857,Colombia,DF
Costa Rica,DF,27.444444,Costa Rica,DF
Croatia,DF,27.125000,Croatia,DF
Denmark,DF,26.285714,Denmark,DF
Egypt,DF,28.750000,Egypt,DF


In [ ]:
# df.insert(2, 'Team1_DF', df['Team1'].map(grouped_wc_fifa18_stats.set_index('Team_name','pos')['Age']))

In [ ]:
#AVG Caps of each team per position
# squads.groupby(["Team","Position"])['Caps'].sum()/squads.groupby(["Team","Position"])['Caps'].size()


# change Rounds

In [ ]:
rounds = df.Rounds
rounds.unique()
rounds_dict = {"Group Phase":1, "Round of 16":2, "Quarter Finals":3, "Semi Finals":4, "Play-off for third place":5, "Finals":6}
rounds_encoded = []

for i in range(len(df)):
    rounds_encoded.append(rounds_dict[list(df.Rounds)[i]])
    
df["Rounds"] = rounds_encoded

In [ ]:
df.head()

# Player Features

In [ ]:
fifa18 = fifa18.replace({"Korea Republic":"South Korea"})
wc_player_stats_18 = fifa18[fifa18["name"].isin(squads.Player) | fifa18["full_name"].isin(squads.Player)]
wc_fifa18_stats = fifa18[fifa18.nationality.isin(squads.Team)]

In [ ]:
wc_fifa18_stats.replace([False,"False",True,"True"],[0,0,1,1])
unused_var=[]
for i in fifa18.columns:
    if "prefers" in i:
        unused_var.append(i)
wc_fifa18_stats = wc_fifa18_stats.drop(["club_logo","ID","real_face","birth_date","flag","photo"],axis=1)
wc_fifa18_stats = wc_fifa18_stats.drop(unused_var,axis=1)
grouped_wc_fifa18_stats = wc_fifa18_stats.groupby(['nationality']).apply(lambda x: (x.sort_values('overall',ascending=False)).head(30).mean()).sort_values('potential',ascending=True)
grouped_wc_fifa18_stats.head()

In [ ]:
grouped_wc_fifa18_stats['Team_name'] = grouped_wc_fifa18_stats.index.get_level_values(0).tolist()

In [ ]:
grouped_wc_fifa18_stats = grouped_wc_fifa18_stats.replace({"Iran":"IR Iran"})
grouped_wc_fifa18_stats.head()

In [ ]:
df.head() 

In [ ]:
for key in grouped_wc_fifa18_stats.keys():
    if key != 'Team_name':
        df.insert(12, 'Team2_'+key, df['Team2'].map(grouped_wc_fifa18_stats.set_index('Team_name')[key]))
        df.insert(12, 'Team1_'+key, df['Team1'].map(grouped_wc_fifa18_stats.set_index('Team_name')[key]))
        

In [ ]:
df.head()

# code country name

In [ ]:
countries = list(set(df["Team1"]).union(set(df["Team2"])))
countries_dict = {}
for i in range(len(countries)):
    countries_dict[countries[i]] = i
    
Team1_encoded = []
Team2_encoded = []

for i in range(len(df)):
    Team1_encoded.append(countries_dict[list(df.Team1)[i]])
    Team2_encoded.append(countries_dict[list(df.Team2)[i]])
    
df["Team1"] = Team1_encoded
df["Team2"] = Team2_encoded

In [ ]:
#Adding goal difference and establishing who is the winner 
winner = []
for i in range (len(df['Team1'])):
    if df['Score1'][i] > df['Score2'][i]:
        winner.append(df['Team1'][i])
    elif df['Score1'][i] < df['Score2'][i]:
        winner.append(df['Team2'][i])
    else:
        winner.append(-1)
df['winning_team'] = winner

df.head()

In [ ]:

df = df.drop(['Score1','Score2','Remarks',"Stadium","City"],axis=1)
df.head(10)

In [ ]:
df.head(30)

# train & test

In [ ]:
X_train = df.loc[df['Rounds'] == 1]
X_test = df.loc[df['Rounds'] != 1]



In [ ]:
y_train = X_train["winning_team"]
y_test = X_test["winning_team"]

In [ ]:
X_train = X_train.drop(["Rounds","winning_team","Time"],axis=1)
X_test = X_test.drop(["Rounds","winning_team","Time"],axis=1)

In [ ]:
X_train.head()

In [ ]:
df.head()

In [ ]:
df.to_csv(".\out.csv")

In [ ]:
# clf = DecisionTreeClassifier()
# clf.fit (X_train, y_train)

# Pred_target = clf.predict(X_test)
# accuracy_score(y_test,Pred_target)

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=df.shape[1]-3, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
#model.dropout()
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #categorical crossentropy?

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), shuffle=True, epochs=100, batch_size=64)